# Which tasks perform well compared to baseline

## Comparison with baseline spreadsheet

Baseline from single task fine-tuning spreadsheet

https://docs.google.com/spreadsheets/d/1BcwEU1e2dbEOMDbWHSP3zh7B8BBWK6kJJBhaS2hDRX8/edit#gid=1653761555

In [4]:
import pandas as pd
# Baseline from single task fine-tuning spreadsheet:
# https://docs.google.com/spreadsheets/d/1BcwEU1e2dbEOMDbWHSP3zh7B8BBWK6kJJBhaS2hDRX8/edit#gid=1653761555

sheet_id = "1BcwEU1e2dbEOMDbWHSP3zh7B8BBWK6kJJBhaS2hDRX8"
sheet_name = "1653761555"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_performance = pd.read_csv(url)

df_performance = df_performance.iloc[:, 1:6]
df_performance.columns = ['task_name_metric', 't5_baseline', 'd2', 
                          'baseline_better', 'multi_task_better']
df_performance.reset_index(drop=True, inplace=True)
task_name = df_performance.iloc[:, 0]
df_performance['task_name'] = [x.split('/')[0] for x in task_name]
df_performance['metric'] = [x.split('/')[1] for x in task_name]
df_performance['multi_task_better_by'] = [x-y for x, y in zip(df_performance['d2'], 
                                                              df_performance['t5_baseline'])]

df_performance = df_performance.dropna()
df_performance.reset_index(drop=True, inplace=True)
df_performance

print(set(df_performance['metric']))

df_performance

{'sequence_accuracy'}


,task_name_metric,t5_baseline,d2,baseline_better,multi_task_better,task_name,metric,multi_task_better_by
0,ai2_arc_ARC_Challenge_false/sequence_accuracy,0.000,0.000,False,False,ai2_arc_ARC_Challenge_false,sequence_accuracy,0.000
1,ai2_arc_ARC_Challenge_qa_options/sequence_accu...,27.425,40.803,False,True,ai2_arc_ARC_Challenge_qa_options,sequence_accuracy,13.378
2,ai2_arc_ARC_Challenge_test/sequence_accuracy,26.421,48.829,False,True,ai2_arc_ARC_Challenge_test,sequence_accuracy,22.408
3,ai2_arc_ARC_Easy_false/sequence_accuracy,0.000,0.000,False,False,ai2_arc_ARC_Easy_false,sequence_accuracy,0.000
4,ai2_arc_ARC_Easy_qa_options/sequence_accuracy,37.193,60.000,False,True,ai2_arc_ARC_Easy_qa_options,sequence_accuracy,22.807
...,...,...,...,...,...,...,...,...
64,super_glue_wsc.fixed_Here_p_stands_for_/sequen...,30.769,28.846,True,False,super_glue_wsc.fixed_Here_p_stands_for_,sequence_accuracy,-1.923
65,super_glue_wsc.fixed_In_the_previous_sentence_...,30.769,28.846,True,False,super_glue_wsc.fixed_In_the_previous_sentence_...,sequence_accuracy,-1.923
66,super_glue_wsc.fixed_Who_is_are_/sequence_accu...,31.731,31.731,False,False,super_glue_wsc.fixed_Who_is_are_,sequence_accuracy,0.000
67,super_glue_wsc.fixed_in_the_passage_above_the_...,33.654,28.846,True,False,super_glue_wsc.fixed_in_the_passage_above_the_...,sequence_accuracy,-4.808


## Graphics

Groups of well and poorly performing tasks

In [5]:
df_performance['task_name_prefix'] = [x.split('_')[0] 
                                      if x.split('_')[0] != 'super'
                                      else '_'.join(x.split('_')[0:3]) 
                                      for x in df_performance['task_name']]

task_group = []
sg_poor_perf = 'super_glue.(wsc.fixed_wic_cb)'
for p in df_performance['task_name_prefix']:
    if p in ['super_glue_wsc.fixed', 'super_glue_wic', 'super_glue_cb']:
        task_group.append(sg_poor_perf)
    elif p != 'hans':
        task_group.append('other')
    else:
        task_group.append(p)

df_performance['task_group'] = task_group

In [6]:
import plotly.express as px

df_plot = df_performance[["task_name", "multi_task_better_by", "task_group"]]
df_plot.columns = ["Task", "Experiment D multi-task training gain (metric)", "group"]

fig = px.bar(df_plot, x=df_plot.columns[0], y=df_plot.columns[1],
             color=df_plot.columns[2], title="Comparison with t5-base") #, orientation='h')
fig.update_xaxes(showticklabels=False)             
fig.show()